In [1]:
import pandas as pd
import numpy as np
import sys
#import sources.endomondolib as endo
#import sources.pysparkconvenience as ps
from numpy import array
from math import sqrt
from pyspark.sql.functions import *
from pyspark.mllib.clustering import KMeans, KMeansModel
from pyspark.sql import DataFrameReader
from pyspark.sql import SQLContext
from IPython.display import display, HTML
from pyspark.sql.functions import col, mean, min, max
from pyspark.ml.linalg import Vectors
from pyspark.ml import Pipeline
from pyspark.ml.regression import LinearRegression, DecisionTreeRegressor, GBTRegressor, RandomForestRegressor
from pyspark.ml.regression import LinearRegressionModel, DecisionTreeRegressionModel, GBTRegressionModel, RandomForestRegressionModel
from pyspark.ml.feature import StringIndexer, VectorIndexer
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import MulticlassClassificationEvaluator, RegressionEvaluator

from pyspark.ml.feature import VectorAssembler

# Disable warnings, set Matplotlib inline plotting and load Pandas package
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline
pd.options.display.mpl_style = 'default'

%load_ext autotime

In [2]:
def vectorizeData(data):
        return data.rdd.map(lambda r: [r[0], r[1], r[2], r[3], Vectors.dense(r[4:-1]),float(r[-1])])\
            .toDF(['route cluster', 'performance cluster', 'userid', 'workoutid', 'features', 'label'])

time: 2.37 ms


In [3]:
#sc = SQLContext()
#create pyspark dataframe from csv
def df_from_csv(csv_file):
    text = sc.textFile(csv_file)\
       .map(lambda line: line.split(','))
   #didn’t work with take(1). believe returns
   #different object then first()
    schema = text.first()
    data = text.filter(lambda x: x != schema)
    df = sqlContext.createDataFrame(data, schema)
    return df

time: 3.94 ms


In [4]:
!ls

autotime.py		   pats_run_results.csv
autotime.pyc		   Regression Results Analysis.ipynb
derby.log		   Regression with Model Saving.ipynb
fit_model_df.csv	   results_df.csv
fit_model_evaluations.csv  rmse.html
Models			   Stacking_Models
Pat_Prediction.ipynb	   Stacking Playground.ipynb
pats_run.csv		   test_save
Pats_run_prediction.csv
time: 110 ms


In [5]:
pats_run_df = pd.read_csv('pats_run.csv')
df = sqlContext.createDataFrame(pats_run_df)

time: 1.57 s


In [6]:
df.show(5)

+----------+---------+----------------+---------------+------+-------------+--------------+--------------+-----------+------------+------------------+-------------------+
|Unnamed: 0|workoutid|route_prediction|perf_prediction|userid|diff_altitude|  geo_distance|heart_rate_avg|  speed_avg|elapsed_time|    user_avg_speed|      user_avg_dist|
+----------+---------+----------------+---------------+------+-------------+--------------+--------------+-----------+------------+------------------+-------------------+
|         0|871971752|               0|              2|212295|         16.3|0.161706242812|             0|11.96088883|     4844.59|10.025832880518179|0.14012889348074767|
+----------+---------+----------------+---------------+------+-------------+--------------+--------------+-----------+------------+------------------+-------------------+

time: 2.29 s


In [7]:
select_columns = ['route_prediction', 'perf_prediction','userid', 'workoutid', 'geo_distance', 'diff_altitude', \
                'user_avg_dist', 'user_avg_speed', 'elapsed_time']

df_for_pred = vectorizeData(df.select(select_columns))

time: 658 ms


In [8]:
fit_model_df = pd.DataFrame.from_csv('fit_model_evaluations.csv')

time: 3.59 ms


In [9]:
fit_model_df[fit_model_df['route cluster'] == 0]

,route cluster,performance cluster,index,path,mae,r2,rmse,cluster mean,cluster standard deviation,std - rmse,std - mae,% drop of std - rmse vs mean,% drop of std - mae vs mean,% std,% rmse,% mae,model type
5,0,1,Route0_Perf1_gbt,/home/ubuntu/Regression_5th_Iteration/Route0_P...,432.063304,0.598188,592.279257,3675.835313,934.374180,342.094923,502.310876,0.093066,0.136652,0.254194,0.161128,0.117542,gbt
1,0,0,Route0_Perf0_gbt,/home/ubuntu/Regression_5th_Iteration/Route0_P...,431.621336,0.556981,591.385644,3555.609900,888.516473,297.130829,456.895137,0.083567,0.128500,0.249891,0.166325,0.121392,gbt
9,0,2,Route0_Perf2_gbt,/home/ubuntu/Regression_5th_Iteration/Route0_P...,453.266263,0.595519,630.684104,3760.843542,991.693316,361.009211,538.427053,0.095992,0.143167,0.263689,0.167698,0.120522,gbt
3,0,0,Route0_Perf0_rfr,/home/ubuntu/Regression_5th_Iteration/Route0_P...,526.111130,0.392530,692.503826,3555.609900,888.516473,196.012647,362.405343,0.055128,0.101925,0.249891,0.194764,0.147966,rfr
7,0,1,Route0_Perf1_rfr,/home/ubuntu/Regression_5th_Iteration/Route0_P...,546.291741,0.410981,717.100507,3675.835313,934.374180,217.273672,388.082439,0.059109,0.105577,0.254194,0.195085,0.148617,rfr
4,0,1,Route0_Perf1_dt,/home/ubuntu/Regression_5th_Iteration/Route0_P...,545.740507,0.407843,719.008217,3675.835313,934.374180,215.365963,388.633672,0.058590,0.105727,0.254194,0.195604,0.148467,dt
0,0,0,Route0_Perf0_dt,/home/ubuntu/Regression_5th_Iteration/Route0_P...,529.494504,0.384126,697.277770,3555.609900,888.516473,191.238703,359.021969,0.053785,0.100973,0.249891,0.196106,0.148918,dt
2,0,0,Route0_Perf0_lr,/home/ubuntu/Regression_5th_Iteration/Route0_P...,544.488171,0.354878,713.642488,3555.609900,888.516473,174.873985,344.028302,0.049183,0.096756,0.249891,0.200709,0.153135,lr
6,0,1,Route0_Perf1_lr,/home/ubuntu/Regression_5th_Iteration/Route0_P...,566.879338,0.372516,740.144526,3675.835313,934.374180,194.229653,367.494841,0.052840,0.099976,0.254194,0.201354,0.154218,lr
11,0,2,Route0_Perf2_rfr,/home/ubuntu/Regression_5th_Iteration/Route0_P...,576.398601,0.403789,765.707938,3760.843542,991.693316,225.985378,415.294715,0.060089,0.110426,0.263689,0.203600,0.153263,rfr


time: 34.2 ms


In [13]:
#load cluster csv
pandas_df = pd.read_csv('/home/ubuntu/endo_sample_6_2_1.csv')
df = sqlContext.createDataFrame(pandas_df)

select_columns = ['route_prediction', 'perf_prediction','userid', 'workoutid', 'geo_distance', 'diff_altitude', \
                'user_avg_dist', 'user_avg_speed', 'elapsed_time']

reg_df = vectorizeData(df.select(select_columns))

route = 0
perf = 2

#model_dictionary
model_dict = {}
model_dict['lr'] = {'model': LinearRegression(featuresCol="features", labelCol="label"),\
                    'param': "ParamGridBuilder() \
                    .addGrid(temp_lr.maxIter, [5, 10]) \
                    .addGrid(temp_lr.regParam, [0, 0.1, 1,10]) \
                    .addGrid(temp_lr.elasticNetParam, [0,0.5,1]) \
                    .build()"}

model_dict['dt'] = {'model': DecisionTreeRegressor(featuresCol="features", labelCol="label",maxMemoryInMB=1028),\
                    'param': "ParamGridBuilder() \
                    .addGrid(temp_lr.maxDepth, [3, 5]) \
                    .addGrid(temp_lr.minInfoGain, [0, 0.1, 1]) \
                    .build()"}

model_dict['gbt'] = {'model': GBTRegressor(featuresCol="features", labelCol="label", maxMemoryInMB=2056),\
                    'param': "ParamGridBuilder() \
                    .addGrid(temp_lr.maxDepth, [3, 5]) \
                    .addGrid(temp_lr.maxIter, [10,20,40]) \
                    .build()"}

model_dict['rfr'] = {'model': RandomForestRegressor(featuresCol="features", labelCol="label", maxMemoryInMB = 2056),\
                    'param': "ParamGridBuilder() \
                    .addGrid(temp_lr.maxDepth, [3, 5]) \
                    .addGrid(temp_lr.numTrees, [10,20,40]) \
                    .build()"}

time: 28.7 s


In [14]:
route_clusters = reg_df.select('route cluster').distinct().collect()
perf_clusters = reg_df.select('performance cluster').distinct().collect()
route_cluster_numbers = [int(route_clusters[i][0]) for i in range(len(route_clusters))]
perf_cluster_numbers = [int(perf_clusters[i][0]) for i in range(len(perf_clusters))]

time: 3.6 s


In [15]:
model_dict.keys()

['rfr', 'dt', 'lr', 'gbt']

time: 2.36 ms


In [16]:
predictions = {}

time: 2.99 ms


In [ ]:
reg_df[(reg_df['route cluster'] == route) & (reg_df['performance cluster'] == 0)].select('features')

DataFrame[features: vector]

time: 61.1 ms


In [ ]:
for m in model_dict.keys():
    for n in perf_cluster_numbers:
        temp_df = reg_df[(reg_df['route cluster'] == route) & (reg_df['performance cluster'] == n)]
        #temp_df1 = reg_df[reg_df['route cluster']==i]
        #temp_df = temp_df1[reg_df['performance cluster']==n]
        temp_df_cv = temp_df


        temp_lr = model_dict[m]['model']
        #temp_lrModel = temp_lr.fit(temp_df['label','features'])
        #temp_df = temp_lrModel.transform(temp_df)

        paramGrid = eval(model_dict[m]['param'])
        #paramGrid = ParamGridBuilder() \
        #.addGrid(temp_lr.maxDepth, [3, 5]) \
        #.addGrid(temp_lr.numTrees, [10,20,40]) \
        #.build()


        evaluator = RegressionEvaluator(
            labelCol="label", predictionCol="prediction", metricName="mae")

        crossval = CrossValidator(estimator=temp_lr,
                                  estimatorParamMaps=paramGrid,
                                  evaluator=evaluator,
                                  numFolds=10)  

        # Run cross-validation, and choose the best set of parameters.
        cvModel = crossval.fit(temp_df_cv)

        pred = cvModel.transform(df_for_pred)
        predictions['Route%d_Perf%d_%s' % (route,n,m)] = pred.collect()[0]['prediction']
        
        print(cvModel.bestModel)
        print(pred.collect()[0]['prediction'])
        #print(cvModel.bestModel.weights)
        #print(cvModel.bestModel.coefficients)
        #basePath = "/nfs/Regression_5th_Iteration/Models/"
        #cvModel.bestModel.save(basePath +"Route%d_Perf%d_%s" % (route,n,m))


RandomForestRegressionModel (uid=rfr_07c822e8ea05) with 10 trees
4337.07587444
RandomForestRegressionModel (uid=rfr_afb0deeb6536) with 20 trees
4157.74448082
RandomForestRegressionModel (uid=rfr_0c7f4ec839cc) with 20 trees
4307.5369603
DecisionTreeRegressionModel (uid=DecisionTreeRegressor_4c9e928060a523d500bd) of depth 5 with 63 nodes
4354.36585366


In [24]:
df_for_pred.collect()[0]['features']

DenseVector([0.1617, 16.3, 0.1401, 10.0258])

time: 190 ms


In [25]:
predictions

{'Route0_Perf0_dt': 4354.365853658536,
 'Route0_Perf0_gbt': 4209.290215692983,
 'Route0_Perf0_lr': 3980.1753294565538,
 'Route0_Perf0_rfr': 4337.075874436801,
 'Route0_Perf1_dt': 4362.796153846154,
 'Route0_Perf1_gbt': 4235.363481009007,
 'Route0_Perf1_lr': 3889.3321480597847,
 'Route0_Perf1_rfr': 4157.744480819445,
 'Route0_Perf2_dt': 4264.911016949152,
 'Route0_Perf2_gbt': 4287.823380825726,
 'Route0_Perf2_lr': 4219.455707819292,
 'Route0_Perf2_rfr': 4307.536960304069}

time: 3.01 ms


In [26]:
pandas_pred = pd.DataFrame.from_dict(predictions, orient = 'index')

time: 1.41 ms


In [51]:
pandas_pred.reset_index()[0]

0     4354.365854
1     4264.911017
2     4287.823381
3     4362.796154
4     4219.455708
5     3889.332148
6     4235.363481
7     4307.536960
8     4337.075874
9     4209.290216
10    4157.744481
11    3980.175329
Name: 0, dtype: float64

time: 4.54 ms


In [ ]:
pandas_pred.to_csv('pats_run_results.csv')

In [52]:
pandas_pred2 = pd.DataFrame()
pandas_pred2['route'] = pandas_pred.reset_index()['index'].str[5]
pandas_pred2['perf'] = pandas_pred.reset_index()['index'].str[11]
pandas_pred2['model_type'] = pandas_pred.reset_index()['index'].str[13:]
pandas_pred2['prediction'] = pandas_pred.reset_index()[0]
pandas_pred2

,route,perf,model_type,prediction
0,0,0,dt,4354.365854
1,0,2,dt,4264.911017
2,0,2,gbt,4287.823381
3,0,1,dt,4362.796154
4,0,2,lr,4219.455708
5,0,1,lr,3889.332148
6,0,1,gbt,4235.363481
7,0,2,rfr,4307.536960
8,0,0,rfr,4337.075874
9,0,0,gbt,4209.290216


time: 19.7 ms


In [55]:
pandas_pred3 = pd.pivot_table(pandas_pred2, values = 'prediction', index = ['route', 'perf'], columns = 'model_type').reset_index()
pandas_pred3


model_type,route,perf,dt,gbt,lr,rfr
0,0,0,4354.365854,4209.290216,3980.175329,4337.075874
1,0,1,4362.796154,4235.363481,3889.332148,4157.744481
2,0,2,4264.911017,4287.823381,4219.455708,4307.536960


time: 17.2 ms


In [57]:
pandas_pred3['label'] = df_for_pred.collect()[0]['label']
pandas_pred3

model_type,route,perf,dt,gbt,lr,rfr,label
0,0,0,4354.365854,4209.290216,3980.175329,4337.075874,4844.59
1,0,1,4362.796154,4235.363481,3889.332148,4157.744481,4844.59
2,0,2,4264.911017,4287.823381,4219.455708,4307.536960,4844.59


time: 203 ms


In [58]:
pandas_pred3.to_csv('pats_run_results3.csv')

time: 1.87 ms


In [2]:
pandas_pred3 = pd.DataFrame.from_csv('pats_run_results3.csv')

time: 4.59 ms


In [3]:
stacking_model = LinearRegressionModel.load('/nfs/Regression_5th_Iteration/Stacking_Models/lr stack/')
new_df = sqlContext.createDataFrame(pandas_pred3)

def vectorize_for_stacking(data):
    return data.rdd.map(lambda r: [r[0], r[1],Vectors.dense(r[2:-1]),float(r[-1])])\
            .toDF(['route', 'perf', 'features', 'label'])

stacking_df = vectorize_for_stacking(new_df)
stacking_df.show()

+-----+----+--------------------+-------+
|route|perf|            features|  label|
+-----+----+--------------------+-------+
|    0|   0|[4354.36585366,42...|4844.59|
|    0|   1|[4362.79615385,42...|4844.59|
|    0|   2|[4264.91101695,42...|4844.59|
+-----+----+--------------------+-------+

time: 9.51 s


In [4]:
stacking_df.collect()[0]

Row(route=0, perf=0, features=DenseVector([4354.3659, 4209.2902, 3980.1753, 4337.0759]), label=4844.59)

time: 210 ms


In [5]:
stacking_df = stacking_model.transform(stacking_df)
stacking_df.show()

+-----+----+--------------------+-------+------------------+
|route|perf|            features|  label|        prediction|
+-----+----+--------------------+-------+------------------+
|    0|   0|[4354.36585366,42...|4844.59|3820.4278529432804|
|    0|   1|[4362.79615385,42...|4844.59|3777.2885825445655|
|    0|   2|[4264.91101695,42...|4844.59| 4179.765046042364|
+-----+----+--------------------+-------+------------------+

time: 923 ms


In [6]:
stacking_pd_df = stacking_df.toPandas()
stacking_pd_df['% error'] = (stacking_pd_df['label'] - stacking_pd_df['prediction'])/stacking_pd_df['label']
stacking_pd_df['abs error'] = (stacking_pd_df['label'] - stacking_pd_df['prediction'])
stacking_pd_df.head()

,route,perf,features,label,prediction,% error,abs error
0,0,0,"[4354.36585366, 4209.29021569, 3980.17532946, ...",4844.59,3820.427853,0.211403,1024.162147
1,0,1,"[4362.79615385, 4235.36348101, 3889.33214806, ...",4844.59,3777.288583,0.220308,1067.301417
2,0,2,"[4264.91101695, 4287.82338083, 4219.45570782, ...",4844.59,4179.765046,0.137230,664.824954


time: 183 ms


In [16]:
stacking_pd_df.columns = ['Route Cluster', 'Performance Cluster', 'Features', 'Actual Value', 'Predicted Value', '% Error', 'Absolute Error']

time: 1.39 ms


In [17]:
stacking_pd_df

,Route Cluster,Performance Cluster,Features,Actual Value,Predicted Value,% Error,Absolute Error
0,0,0,"[4354.36585366, 4209.29021569, 3980.17532946, ...",4844.59,3820.427853,0.211403,1024.162147
1,0,1,"[4362.79615385, 4235.36348101, 3889.33214806, ...",4844.59,3777.288583,0.220308,1067.301417
2,0,2,"[4264.91101695, 4287.82338083, 4219.45570782, ...",4844.59,4179.765046,0.137230,664.824954


time: 13.6 ms


In [19]:
stacking_pd_df[['Route Cluster', 'Performance Cluster', 'Actual Value', 'Predicted Value', '% Error', 'Absolute Error']].set_index(['Route Cluster', 'Performance Cluster'])

Actual Value  Predicted Value   % Error  \
Route Cluster Performance Cluster                                            
0             0                         4844.59      3820.427853  0.211403   
              1                         4844.59      3777.288583  0.220308   
              2                         4844.59      4179.765046  0.137230   

                                   Absolute Error  
Route Cluster Performance Cluster                  
0             0                       1024.162147  
              1                       1067.301417  
              2                        664.824954

time: 14.7 ms


In [9]:
stacking_pd_df.to_csv('run_predictions_for_publication.csv')

time: 1.71 ms


In [ ]:
stacking_pd_df.from_csv('run_predictions_for_publication.csv')